## Grupo 2: Análisis de reservas y cancelaciones hoteleras: patrones y predicciones
🎯 Objetivo del proyecto
Analizar los datos de reservas de hoteles para comprender el perfil de los clientes, identificar patrones de comportamiento (estacionalidad, duración de la estancia, precios) y estudiar los factores que influyen en las cancelaciones, utilizando herramientas de análisis de datos en Python.

### 1. Introducción y descripción del dataset

- Breve explicación del contexto del análisis

- Descripción de las variables disponibles

- Objetivo del estudio

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('hotel_bookings.csv')

### 2. Perfil general de las reservas (Patri)

Objetivo: Entender cómo son las reservas y los clientes en general.

Tareas:

* Limpieza y descripción del dataset: tipos de datos, valores nulos, duplicados.
* Análisis de variables descriptivas:

  * *Tipo de hotel* (city vs. resort).
  * *Número de huéspedes* (adultos, niños, bebés).
  * *Países de origen*: top 10 países.
  * *Canales de reserva* (agencias online, offline, directas).
* Visualizaciones:

  * Gráfico de barras comparando city hotel vs resort hotel.
  * Gráfico circular de canales de reserva.
  * Mapa mundial con procedencia de clientes.

### Preguntas a responder:

* ¿Cuál es el perfil típico de cliente?
* ¿Qué países aportan más reservas?
* ¿Por qué canales se reserva más?

In [ ]:
print("--- Primeras 5 filas ---")
print(df.head())

print("\n--- Información general y tipos de datos ---")
df.info()

print("\n--- Resumen estadístico de las columnas numéricas ---")
print(df.describe())

print("\n--- Cuántos datos nulos hay ---")
print(df.isnull().sum())

# Pero esto es más informativo:
print(f"Total de filas: {len(df)}")
print(f"Filas únicas: {len(df.drop_duplicates())}")

duplicados_df = df[df.duplicated(keep='first')]
print(f"\nPrimeras filas marcadas como duplicadas:")
print(duplicados_df.head())

Mirar por qué con size() sale bien y con value_counts() no

In [ ]:
clientes_hotel_anio = df.groupby(['arrival_date_year', 'hotel']).size().reset_index(name='cantidad_clientes')

sns.barplot(data=clientes_hotel_anio, x='arrival_date_year', y='cantidad_clientes', hue='hotel')
plt.title('Cantidad de clientes por año')
plt.xlabel('Año')
plt.ylabel('Clientes')
plt.show()

In [ ]:
count_paises = df['country'].nunique()

reservas_por_pais = df.groupby('hotel')['country'].value_counts()
print(reservas_por_pais)

# Crear figura
fig, ax = plt.subplots(figsize=(2, 1))
ax.axis('off')  # Ocultar ejes

# Mostrar texto
ax.text(0.5, 0.5, count_paises, ha='center', va='center', fontsize=12, fontweight='bold')

plt.title("Países distintos en total:", fontsize=14)
plt.show()

In [ ]:
num_paises = df.groupby('hotel')['country'].nunique()

sns.barplot(x=num_paises.index, y=num_paises.values)
plt.title('Número de países distintos por hotel')
plt.xlabel('Hotel')
plt.ylabel('Paises')
plt.show()

In [ ]:
import plotly.express as px

for hotel in df['hotel'].unique():
    reservas_hotel = df[df['hotel']==hotel]['country'].value_counts().reset_index()
    reservas_hotel.columns = ['country', 'reservas']
    
    fig = px.choropleth(
        reservas_hotel,
        locations='country',
        color='reservas',
        color_continuous_scale='Blues',
        title=f'Reservas por país - {hotel}'
    )
    fig.show()

In [ ]:
canales = df['distribution_channel'].value_counts()
print(canales)

# Umbral: menos del 5% se agrupa en 'Otros'
umbral = 0.05 * canales.sum()
canales_agrupados = canales.copy()
canales_agrupados[canales < umbral] = 0  # asignar 0 temporalmente

otros = canales[canales < umbral].sum()
canales_agrupados = canales_agrupados[canales_agrupados>0]
canales_agrupados['Otros'] = otros

plt.figure(figsize=(8, 6))
# Crear gráfico circular
wedges, texts, autotexts = plt.pie(
    canales_agrupados.values,
    labels=None,          # etiquetas en la leyenda, no en el gráfico
    autopct='%1.1f%%',
    startangle=90,
    colors=['skyblue','lightgreen','orange','pink'],
    pctdistance=1.1      # porcentaje dentro pero más separado
)

# Agregar leyenda al lado
plt.legend(
    wedges,                 # los "trozos" del pie
    canales_agrupados.index,          # nombres de los canales
    title="Canales de reserva",
    loc="center left",
    bbox_to_anchor=(0.9, 0, 0.5, 1)
)

plt.axis('equal')
plt.show()

### 3. Patrones de comportamiento y cancelaciones (Rodri)

Objetivo: Analizar cómo varían las reservas en función del tiempo, los precios y la estancia.

Tareas:

* Estudiar variables clave:

  * *Mes de llegada* → estacionalidad.
  * *Precio promedio por noche (adr)* → variación según temporada.
  * *Cancelaciones*: proporción general y por segmento.
* Visualizaciones:

  * Línea temporal con reservas por mes.
  * Boxplot de precios según temporada alta/baja.
  * Gráfico de barras: cancelaciones por canal de reserva.

### Preguntas a responder:

* ¿En qué meses hay más reservas y cancelaciones?
* ¿El precio influye en la probabilidad de cancelación?
* ¿Los hoteles de ciudad o de resort tienen más cancelaciones?


Analizamos el número de reservas por mes para determinar la estacionalidad. También analizamos el número de cancelaciones por mes.
¿En qué meses hay más reservas y cancelaciones?

In [ ]:
# Análisis de estacionalidad - Porcentaje de clientes por mes

# Contar reservas por mes
reservas_por_mes = df['arrival_date_month'].value_counts()

# Calcular porcentaje por mes
porcentaje_por_mes = (df['arrival_date_month'].value_counts(normalize=True) * 100).round(2)

# Crear una tabla más visual ORDENADA por mes cronológico
resumen_mes = pd.DataFrame({
    'Reservas': reservas_por_mes,
    'Porcentaje': porcentaje_por_mes
})

# Definir el orden correcto de los meses
orden_meses = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

# Reordenar según el orden cronológico de los meses
resumen_mes = resumen_mes.reindex(orden_meses)

print("=== RESUMEN COMPLETO (ORDENADO CRONOLÓGICAMENTE) ===")
print(resumen_mes)

# Histograma de reservas por mes
plt.figure(figsize=(12, 6))
plt.bar(resumen_mes.index, resumen_mes['Reservas'], color='skyblue', edgecolor='navy', alpha=0.7)
plt.title('Histograma: Distribución de Reservas por Mes', fontsize=16, fontweight='bold')
plt.xlabel('Mes', fontsize=12)
plt.ylabel('Número de Reservas', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

# Añadir valores en las barras
for i, v in enumerate(resumen_mes['Reservas']):
    plt.text(i, v + 100, f'{v:,.0f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Calcular tasa de cancelación por mes (cancelaciones/total_reservas_del_mes), calculamos la tasa para que el resultado sea independiente del numero de cancelaciones
print("=== TASA DE CANCELACIÓN POR MES ===")
tasa_cancelacion_mes = df.groupby('arrival_date_month')['is_canceled'].mean() * 100
tasa_cancelacion_ordenada = tasa_cancelacion_mes.reindex(orden_meses)
print(tasa_cancelacion_ordenada.round(2))

# Visualización de la tasa de cancelación por mes
plt.figure(figsize=(14, 8))

bars = plt.bar(tasa_cancelacion_ordenada.index, tasa_cancelacion_ordenada.values, edgecolor='black', alpha=0.8)

plt.title('Tasa de Cancelación por Mes (%)', fontsize=18, fontweight='bold', pad=20)
plt.xlabel('Mes', fontsize=14, fontweight='bold')
plt.ylabel('Tasa de Cancelación (%)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

# Añadir valores en las barras
for i, (mes, tasa) in enumerate(tasa_cancelacion_ordenada.items()):
    plt.text(i, tasa + 0.5, f'{tasa:.1f}%', ha='center', va='bottom', 
             fontweight='bold', fontsize=11)

# Añadir línea de referencia con la media
media_cancelacion = tasa_cancelacion_ordenada.mean()
plt.axhline(y=media_cancelacion, color='blue', linestyle='--', linewidth=2, 
            label=f'Media: {media_cancelacion:.1f}%')

# Añadir rejilla y leyenda
plt.grid(axis='y', alpha=0.3)
plt.legend(fontsize=12)
plt.show()

Analisis de la variacion del precio con los meses

In [ ]:
# Boxplot: Variación de precios según temporada

# Definir temporadas basándose en los meses
def categorizar_temporada(mes):
    if mes in ['December', 'January', 'February']:
        return 'Invierno'
    elif mes in ['March', 'April', 'May']:
        return 'Primavera'
    elif mes in ['June', 'July', 'August']:
        return 'Verano'
    else: 
        return 'Otoño'

# Crear columna de temporada
df['temporada'] = df['arrival_date_month'].apply(categorizar_temporada)

temporadas_ordenadas = ['Invierno', 'Primavera', 'Verano', 'Otoño']
datos_temporadas = [df[df['temporada'] == temp]['adr'] for temp in temporadas_ordenadas]

# Gráfico de barras con precio promedio
precio_promedio_temp = df.groupby('temporada')['adr'].mean().reindex(temporadas_ordenadas)
barras = plt.bar(temporadas_ordenadas, precio_promedio_temp.values, alpha=0.8 ,edgecolor='black', linewidth=1.5)

# Añadir valores en las barras
for bar, valor in zip(barras, precio_promedio_temp.values):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'${valor:.0f}', ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.title('Precio Promedio por Temporada\n(Gráfico de Barras)', fontsize=14, fontweight='bold')
plt.xlabel('Temporada', fontsize=12, fontweight='bold')
plt.ylabel('Precio Promedio ADR ($)', fontsize=12, fontweight='bold')
plt.tick_params(axis='x', rotation=45)
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()

# Añadir estadísticas por temporada
print("=== ESTADÍSTICAS DE PRECIOS POR TEMPORADA ===")
for temp in ['Primavera', 'Verano', 'Otoño', 'Invierno']:
    precios_temp = df[df['temporada'] == temp]['adr']
    print(f"\n{temp}:")
    print(f"  Precio promedio: ${precios_temp.mean():.2f}")

plt.show()

# Análisis adicional: precio promedio por temporada
precio_por_temporada = df.groupby('temporada')['adr'].mean().sort_values(ascending=False)
print(f"\n=== RANKING DE TEMPORADAS POR PRECIO PROMEDIO ===")
for i, (temp, precio) in enumerate(precio_por_temporada.items(), 1):
    print(f"{i}. {temp}: ${precio:.2f}")

Analisis de la correlacion entre el precio y la tasa de cancelacion ¿El precio influye en la probabilidad de cancelación?

In [ ]:
# ¿El precio influye en la probabilidad de cancelación?

print("=== ANÁLISIS: PRECIO vs CANCELACIÓN ===")

# 1. Estadísticas básicas del precio (ADR - Average Daily Rate)
print("1. ESTADÍSTICAS BÁSICAS DEL PRECIO")

print(f"Precio promedio general: ${df['adr'].mean():.2f}")
print(f"Precio mediano: ${df['adr'].median():.2f}")

# 2. Comparar precios entre reservas canceladas vs no canceladas
precio_canceladas = df[df['is_canceled'] == 1]['adr']
precio_no_canceladas = df[df['is_canceled'] == 0]['adr']

print(f"\nPrecio promedio - Reservas CANCELADAS: ${precio_canceladas.mean():.2f}")
print(f"Precio promedio - Reservas NO canceladas: ${precio_no_canceladas.mean():.2f}")
print(f"Diferencia: ${precio_canceladas.mean() - precio_no_canceladas.mean():.2f}")

# 4. Correlación entre precio y cancelación
print("\n" + "="*50)
print("2. CORRELACIÓN PRECIO-CANCELACIÓN")
correlacion = df['adr'].corr(df['is_canceled'])
print(f"Correlación entre precio (ADR) y cancelación: {correlacion:.4f}")

¿Las cancelaciones varían por tipo de hotel?

In [ ]:
# Calcular cancelaciones por tipo de hotel
print("CANCELACIONES POR TIPO DE HOTEL")

# Cancelaciones absolutas por tipo
cancelaciones_por_tipo = df[df['is_canceled'] == 1]['hotel'].value_counts()
print("Número de cancelaciones por tipo:")
print(cancelaciones_por_tipo)

#  TASA DE CANCELACIÓN por tipo (lo más importante)
print("TASA DE CANCELACIÓN POR TIPO (cancelaciones/total_reservas)")

tasa_por_tipo = df.groupby('hotel').agg({
    'is_canceled': ['count', 'sum', 'mean']
}).round(4)

# Simplificar nombres de columnas
tasa_por_tipo.columns = ['Total_Reservas', 'Total_Cancelaciones', 'Tasa_Cancelacion']
tasa_por_tipo['Tasa_Cancelacion_Pct'] = (tasa_por_tipo['Tasa_Cancelacion'] * 100).round(2)

print(tasa_por_tipo)

print("\n" + "="*60)

# 4. Interpretación y comparación
print("4. INTERPRETACIÓN")
city_hotel_rate = tasa_por_tipo.loc['City Hotel', 'Tasa_Cancelacion_Pct']
resort_hotel_rate = tasa_por_tipo.loc['Resort Hotel', 'Tasa_Cancelacion_Pct']

diferencia = abs(city_hotel_rate - resort_hotel_rate)
print(f"City Hotel - Tasa de cancelación: {city_hotel_rate}%")
print(f"Resort Hotel - Tasa de cancelación: {resort_hotel_rate}%")
print(f"Diferencia: {diferencia:.2f} puntos porcentuales")

if city_hotel_rate > resort_hotel_rate:
    print(f"→ Los CITY HOTELS tienen {diferencia:.2f}% MÁS cancelaciones que los Resort Hotels")
elif resort_hotel_rate > city_hotel_rate:
    print(f"→ Los RESORT HOTELS tienen {diferencia:.2f}% MÁS cancelaciones que los City Hotels")
else:
    print("→ Ambos tipos de hotel tienen tasas de cancelación similares")

### 4. Predicción y recomendaciones (Albert)

Objetivo: Explorar qué factores están más relacionados con las cancelaciones y proponer conclusiones prácticas.

Tareas:

* Crear variables comparativas:

  * Reservas canceladas vs no canceladas.
  * Relación entre lead_time (antelación de reserva) y cancelación.
  * Clientes repetidores vs nuevos.
* Visualizaciones:

  * Heatmap de correlaciones entre variables numéricas y cancelación.
  * Gráfico de barras de cancelación según país.
  * Boxplot de lead_time para canceladas vs no canceladas.
* (Opcional avanzado) Entrenar un modelo simple de clasificación para predecir cancelaciones (ej: logistic regression, random forest).

### Preguntas a responder:

* ¿Qué variables son más importantes para explicar una cancelación?
* ¿Se pueden detectar patrones que ayuden a los hoteles a reducir cancelaciones?
* ¿Qué recomendaciones se pueden dar a un hotel basadas en el análisis?